In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix

import implicit

Читаем клики пользователей

In [5]:
train_clicks = pd.read_csv('train_clicks.csv')
train_clicks[:5]

,user_id,picture_id,day
0,1442,546149,2019-02-09
1,1442,546149,2019-02-09
2,1442,1242875,2019-02-16
3,1442,1242875,2019-02-16
4,1442,1242891,2019-02-16


In [7]:
users = train_clicks.user_id
items = train_clicks.picture_id

Создаём разреженную матрицу пользователь — объект. Используем модель **coo_matrix**, она позволяет быстро сформировать матрицу, зная координаты значений

In [9]:
user_item = coo_matrix((np.ones_like(users), (users, items)))
user_item

<499676x1498020 sparse matrix of type '<class 'numpy.int64'>'
	with 2360862 stored elements in COOrdinate format>

В качестве модели будем использовать разложение матрицы с помощью метода ALS

In [5]:
# model = implicit.als.AlternatingLeastSquares(factors=10, iterations=30)
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=30)

In [6]:
model.fit(user_item.T.tocsr())

100%|██████████| 30.0/30 [01:26<00:00,  2.39s/it]


Прочитаем идентификаторы пользователей, для которых нам нужно сделать предсказания

In [7]:
test_users = pd.read_csv('test_users.csv')

In [8]:
user_item_csr = user_item.tocsr()

Для каждого пользователя найдём 100 самых релевантных изображений

In [9]:
rows = []
for user_id in test_users.user_id.values:
    items = [i[0] for i in model.recommend(user_id, user_item_csr, N=100)]
    rows.append(items)

Отформатируем идентификаторы как нужно для ответа

In [10]:
test_users['predictions'] = list(map(lambda x: ' '.join(map(str, x)), rows))

И запишем предсказания в файл

In [11]:
test_users.to_csv('predictions.csv', index=False)